In [1]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import Span
from bokeh.models.annotations import Title
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, FixedTicker, SingleIntervalTicker, LinearAxis
from bokeh.layouts import gridplot
from bokeh.transform import linear_cmap
from bokeh.palettes import all_palettes
import datetime
from bokeh.io import export_png
from selenium import webdriver
import chromedriver_binary
output_notebook()

Loading BokehJS ...

In [2]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [3]:
infer_schema = "True"
first_row_is_header = "True"
delimiter = ";"
file_type = "csv"
file_location = "ads_produccion.csv"

ads_produccion = (
   spark.read.format(file_type)
    .option("inferSchema", infer_schema)
    .option("header", first_row_is_header)
    .option("sep", ',')
    .load(file_location)
)

# Base de datos `ads_produccion.csv`

In [4]:
ads_produccion = (
    ads_produccion.withColumn('fecha', F.to_timestamp('Fecha_Hora', 'yyyy-MM-dd HH:mm:ss'))
    .where((F.col('ton_total').isNotNull())
          & (F.col('ton_total') > 0))
    .drop('Fecha_Hora', 'n_perfo', 'n_eq_apoyo', 'n_aljibe')      
)

In [5]:
ads_produccion.printSchema()

root
 |-- ton_total: double (nullable = true)
 |-- n_descargas: double (nullable = true)
 |-- n_cam: double (nullable = true)
 |-- n_shov: double (nullable = true)
 |-- ton_chancador: double (nullable = true)
 |-- ton_botadero: double (nullable = true)
 |-- descargas_botadero: double (nullable = true)
 |-- ton_chancador_1: double (nullable = true)
 |-- ton_chancador_2: double (nullable = true)
 |-- cam_chancador: double (nullable = true)
 |-- cam_botadero: double (nullable = true)
 |-- ton_alta_ley: double (nullable = true)
 |-- ton_media_ley: double (nullable = true)
 |-- ton_baja_ley: double (nullable = true)
 |-- ton_lastre: double (nullable = true)
 |-- fecha: timestamp (nullable = true)



In [6]:
for col in ads_produccion.columns:
    nulls = ads_produccion.select(col).where(F.col(col).isNull()).count()
    print(f'Hay {nulls} valores NULL en la columna {col} de un total de {ads_produccion.count()}')

Hay 0 valores NULL en la columna ton_total de un total de 20534
Hay 0 valores NULL en la columna n_descargas de un total de 20534
Hay 0 valores NULL en la columna n_cam de un total de 20534
Hay 0 valores NULL en la columna n_shov de un total de 20534
Hay 0 valores NULL en la columna ton_chancador de un total de 20534
Hay 0 valores NULL en la columna ton_botadero de un total de 20534
Hay 0 valores NULL en la columna descargas_botadero de un total de 20534
Hay 0 valores NULL en la columna ton_chancador_1 de un total de 20534
Hay 0 valores NULL en la columna ton_chancador_2 de un total de 20534
Hay 0 valores NULL en la columna cam_chancador de un total de 20534
Hay 0 valores NULL en la columna cam_botadero de un total de 20534
Hay 0 valores NULL en la columna ton_alta_ley de un total de 20534
Hay 0 valores NULL en la columna ton_media_ley de un total de 20534
Hay 0 valores NULL en la columna ton_baja_ley de un total de 20534
Hay 0 valores NULL en la columna ton_lastre de un total de 20534

In [7]:
for col in ads_produccion.columns:
    try:
        lt_zero = ads_produccion.select(col).where(F.col(col) <= 0).count()
        print(f'Hay {lt_zero} valores negativos en la columna {col} de un total de {ads_produccion.count()}')
    except:
        print(f'La columna {col} no se le puede aplicar esta funcion ya que es de tipo {dict(ads_produccion.dtypes)[col]}')

Hay 0 valores negativos en la columna ton_total de un total de 20534
Hay 0 valores negativos en la columna n_descargas de un total de 20534
Hay 0 valores negativos en la columna n_cam de un total de 20534
Hay 0 valores negativos en la columna n_shov de un total de 20534
Hay 1687 valores negativos en la columna ton_chancador de un total de 20534
Hay 150 valores negativos en la columna ton_botadero de un total de 20534
Hay 120 valores negativos en la columna descargas_botadero de un total de 20534
Hay 2995 valores negativos en la columna ton_chancador_1 de un total de 20534
Hay 3763 valores negativos en la columna ton_chancador_2 de un total de 20534
Hay 1685 valores negativos en la columna cam_chancador de un total de 20534
Hay 120 valores negativos en la columna cam_botadero de un total de 20534
Hay 17375 valores negativos en la columna ton_alta_ley de un total de 20534
Hay 867 valores negativos en la columna ton_media_ley de un total de 20534
Hay 14495 valores negativos en la columna 

In [8]:
def plot_rango_fecha(df, x, y, date_column, fecha_inicio, x_label='', y_label='', extra_info=[], fecha_termino=None, span=True, cmap=True):
    if fecha_termino is None:
        df = (
            df.select('*')
            .where((F.year(F.col(date_column)) == fecha_inicio.split('-')[0])
                    & (F.month(F.col(date_column))== fecha_inicio.split('-')[1]) 
                    & (F.dayofmonth(F.col(date_column)) == fecha_inicio.split('-')[2]))
            .withColumn('h', F.hour(F.col(date_column)))).toPandas()
        title = f'Gráfico Comparativo {fecha_inicio}'
    else:
        df = (
            df.select('*')
            .filter(F.col(date_column).between(datetime.datetime.strptime(f'{fecha_inicio} 00:00:00', '%Y-%m-%d %H:%M:%S'),
                                               datetime.datetime.strptime(f'{fecha_termino} 23:00:00', '%Y-%m-%d %H:%M:%S')))
            .withColumn('h', F.hour(F.col(date_column)))).toPandas()
        title = f'Rango desde {fecha_inicio} a {fecha_termino}'
    x_plot = df[x]
    y_plot = df[y]

    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column], 'h': df['h']}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, '@date{%Y-%m-%d %H:%M}'), (y, '@y{int}'), (x, '@x{int}')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips, formatters={'@date': 'datetime'})
    
    mapper = linear_cmap(field_name='h', palette=all_palettes['PiYG'][4], low=0, high=23)
    color_bar = ColorBar(color_mapper=mapper['transform'], width = 20, ticker=FixedTicker(ticks=np.arange(0, 36, 6), desired_num_ticks=5), scale_alpha=0.7)
    
    p = figure(width=500, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=10, color= (mapper if cmap else '#2171b5') , alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = title
    p.title.text_font_size = '14pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label
        
    if span:
        vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.7)
        hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.7)
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)
    if cmap:
        p.add_layout(color_bar, 'right')

    return p

In [9]:
def datos_diarios(dataframe, col, date_column, fecha_inicio, second_col=False, second_col_name=''):
    results = (
        dataframe.select('*')
        .where((F.year(F.col(date_column)) == fecha_inicio.split('-')[0])
                & (F.month(F.col(date_column))== fecha_inicio.split('-')[1]) 
                & (F.dayofmonth(F.col(date_column)) == fecha_inicio.split('-')[2]))
        .groupBy(F.hour(F.col(date_column)).alias('Time')))
    first_c = (
        results.sum(col)
        .select(f'sum({col})')
        .orderBy('Time')
        .rdd.flatMap(lambda x: x)
        .collect())
    if second_col:
        second_c = (
            results.sum(second_col_name)
            .select(f'sum({second_col_name})')
            .orderBy('Time')
            .rdd.flatMap(lambda x: x)
            .collect())
        data = [(f'{fecha_inicio} {str(i).zfill(2)}:00:00', first_c[i], second_c[i]) for i in range(0, 24)]
        return spark.createDataFrame(data=data, schema=['FECHA', f'{col}', f'{second_col_name}']).withColumn('fecha', F.to_timestamp('FECHA', 'yyyy-MM-dd HH:mm:ss'))
    else:
        data = [(f'{fecha_inicio} {str(i).zfill(2)}:00:00', first_c[i]) for i in range(0, 24)]
        return spark.createDataFrame(data=data, schema=['FECHA', f'{col}']).withColumn('fecha', F.to_timestamp('FECHA', 'yyyy-MM-dd HH:mm:ss'))

In [10]:
show(plot_rango_fecha(df=ads_produccion, date_column='fecha', x='n_cam', y='ton_total', fecha_inicio='2021-11-14', span=True, fecha_termino=None, cmap=True))

In [11]:
show(plot_rango_fecha(df=ads_produccion, date_column='fecha', x='n_shov', y='ton_total', fecha_inicio='2021-11-14', span=False, fecha_termino='2021-12-30', cmap=False))

In [12]:
show(plot_rango_fecha(ads_produccion, date_column='fecha', x='n_descargas', y='ton_total', fecha_inicio='2021-11-14', span=False, fecha_termino='2021-12-30'))

# Base de datos `Datos_movimiento_mina.csv`

In [13]:
infer_schema = "True"
first_row_is_header = "True"
delimiter = ";"
file_type = "csv"
file_location = "Datos_movimiento_mina.csv"
movimiento_mina = (
   spark.read.format(file_type)
    .option("inferSchema", infer_schema)
    .option("header", first_row_is_header)
    .option("sep", delimiter)
    .load(file_location)
)

In [14]:
movimiento_mina.printSchema()

root
 |-- CAEX: string (nullable = true)
 |-- PALA: string (nullable = true)
 |-- shiftindex: integer (nullable = true)
 |-- TONELAJE FC: string (nullable = true)
 |-- # Baldes: string (nullable = true)
 |-- TIEMPO DE ACULATAMIENTO: string (nullable = true)
 |-- TIEMPO DE CARGA: string (nullable = true)
 |-- TIEMPO DE DESCARGA: string (nullable = true)
 |-- TIEMPO ESPERA PALA: string (nullable = true)
 |-- TIEMPO COLA CAMION: string (nullable = true)
 |-- idletime: string (nullable = true)
 |-- dumpingtim: string (nullable = true)
 |-- TIEMPO VIAJE HACIA LA PALA: string (nullable = true)
 |-- TIEMPO DE VIAJE HACIA DESCARGA: string (nullable = true)
 |-- TIEMPO DE CICLO TOTAL CAEX: string (nullable = true)
 |-- TIEMPO DE CICLO EFECTIVO PALA: string (nullable = true)
 |-- DISTANCIA DE VIAJE VACIO: string (nullable = true)
 |-- DISTANCIA DE VIAJE LLENO: string (nullable = true)
 |-- DISTANCIA EQUIVALENTE DE VIAJE VACIO: string (nullable = true)
 |-- DISTANCIA EQUIVALENTE DE VIAJE LLENO: s

## Preprocesamiento

In [15]:
movimiento_mina = (
    movimiento_mina.select('CAEX',
     'PALA',
     F.col('TONELAJE FC').cast(T.DoubleType()),
     F.col('# Baldes').cast(T.DoubleType()),
     F.col('TIEMPO DE ACULATAMIENTO').cast(T.DoubleType()),
     F.col('TIEMPO DE CARGA').cast(T.DoubleType()),
     F.col('TIEMPO DE DESCARGA').cast(T.DoubleType()),
     F.col('TIEMPO ESPERA PALA').cast(T.DoubleType()),
     F.col('TIEMPO COLA CAMION').cast(T.DoubleType()),
     F.col('TIEMPO VIAJE HACIA LA PALA').cast(T.DoubleType()),
     F.col('TIEMPO DE VIAJE HACIA DESCARGA').cast(T.DoubleType()),
     F.col('TIEMPO DE CICLO TOTAL CAEX').cast(T.DoubleType()),
     F.col('TIEMPO DE CICLO EFECTIVO PALA').cast(T.DoubleType()),
     F.col('DISTANCIA DE VIAJE VACIO').cast(T.DoubleType()),
     F.col('DISTANCIA DE VIAJE LLENO').cast(T.DoubleType()),
     F.col('DISTANCIA EQUIVALENTE DE VIAJE VACIO').cast(T.DoubleType()),
     F.col('DISTANCIA EQUIVALENTE DE VIAJE LLENO').cast(T.DoubleType()),            
     'LoadingTimestamp',
     'DumpingTimestamp').withColumnRenamed('TIEMPO DE CARGA', 't_carga')
    .withColumnRenamed('TONELAJE FC', 'ton_camion')
    .withColumnRenamed('# Baldes', 'n_baldes')
    .withColumnRenamed('TIEMPO DE ACULATAMIENTO', 't_aculatamiento')
    .withColumnRenamed('TIEMPO DE DESCARGA', 't_descarga')
    .withColumnRenamed('TIEMPO ESPERA PALA', 't_espera_pala')
    .withColumnRenamed('TIEMPO COLA CAMION', 't_cola_camion')
    .withColumnRenamed('TIEMPO VIAJE HACIA LA PALA', 't_a_pala')
    .withColumnRenamed('TIEMPO DE VIAJE HACIA DESCARGA', 't_viaje_descarga')
    .withColumnRenamed('TIEMPO DE CICLO TOTAL CAEX', 't_ciclo_caex')
    .withColumnRenamed('TIEMPO DE CICLO EFECTIVO PALA', 't_ciclo_pala')
    .withColumnRenamed('DISTANCIA DE VIAJE VACIO', 'd_viaje_vacio')
    .withColumnRenamed('DISTANCIA DE VIAJE LLENO', 'd_viaje_lleno')
    .withColumnRenamed('DISTANCIA EQUIVALENTE DE VIAJE VACIO', 'd_eq_vacio')
    .withColumnRenamed('DISTANCIA EQUIVALENTE DE VIAJE LLENO', 'd_eq_lleno')
    .withColumn('Loading', F.to_timestamp('LoadingTimestamp', 'yyyy-MM-dd HH:mm:ss.SSS'))
    .withColumn('Dumping', F.to_timestamp('DumpingTimestamp', 'yyyy-MM-dd HH:mm:ss.SSS'))
    .drop('LoadingTimestamp', 'DumpingTimestamp')
)
movimiento_mina = movimiento_mina.select([F.when(F.col(c)=="NULL", None).otherwise(F.col(c)).alias(c) for c in movimiento_mina.columns])

In [16]:
movimiento_mina.printSchema()

root
 |-- CAEX: string (nullable = true)
 |-- PALA: string (nullable = true)
 |-- ton_camion: double (nullable = true)
 |-- n_baldes: double (nullable = true)
 |-- t_aculatamiento: double (nullable = true)
 |-- t_carga: double (nullable = true)
 |-- t_descarga: double (nullable = true)
 |-- t_espera_pala: double (nullable = true)
 |-- t_cola_camion: double (nullable = true)
 |-- t_a_pala: double (nullable = true)
 |-- t_viaje_descarga: double (nullable = true)
 |-- t_ciclo_caex: double (nullable = true)
 |-- t_ciclo_pala: double (nullable = true)
 |-- d_viaje_vacio: double (nullable = true)
 |-- d_viaje_lleno: double (nullable = true)
 |-- d_eq_vacio: double (nullable = true)
 |-- d_eq_lleno: double (nullable = true)
 |-- Loading: timestamp (nullable = true)
 |-- Dumping: timestamp (nullable = true)



In [17]:
movimiento_mina.count()

731156

In [18]:
for col in movimiento_mina.columns:
    nulls = movimiento_mina.select(col).where(F.col(col).isNull()).count()
    print(f'Hay {nulls} valores NULL en la columna {col} de un total de {movimiento_mina.count()}')

Hay 0 valores NULL en la columna CAEX de un total de 731156
Hay 1 valores NULL en la columna PALA de un total de 731156
Hay 30485 valores NULL en la columna ton_camion de un total de 731156
Hay 6343 valores NULL en la columna n_baldes de un total de 731156
Hay 158901 valores NULL en la columna t_aculatamiento de un total de 731156
Hay 6344 valores NULL en la columna t_carga de un total de 731156
Hay 3 valores NULL en la columna t_descarga de un total de 731156
Hay 6344 valores NULL en la columna t_espera_pala de un total de 731156
Hay 6344 valores NULL en la columna t_cola_camion de un total de 731156
Hay 63797 valores NULL en la columna t_a_pala de un total de 731156
Hay 12437 valores NULL en la columna t_viaje_descarga de un total de 731156
Hay 6347 valores NULL en la columna t_ciclo_caex de un total de 731156
Hay 6344 valores NULL en la columna t_ciclo_pala de un total de 731156
Hay 30677 valores NULL en la columna d_viaje_vacio de un total de 731156
Hay 21408 valores NULL en la col

#### Hay mas de 158k datos null en la columna `t_aculatamiento`, sin contar otras variables que tienen una gran cantidad de valores `null`

In [19]:
for col in movimiento_mina.columns:
    try:
        lt_zero = movimiento_mina.select(col).where(F.col(col) <= 0).count()
        print(f'Hay {lt_zero} valores negativos o zero en la columna {col} de un total de {movimiento_mina.count()}')
    except:
        print(f'La columna {col} no se le puede aplicar esta funcion ya que es de tipo {dict(movimiento_mina.dtypes)[col]}')

Hay 0 valores negativos o zero en la columna CAEX de un total de 731156
Hay 0 valores negativos o zero en la columna PALA de un total de 731156
Hay 86 valores negativos o zero en la columna ton_camion de un total de 731156
Hay 0 valores negativos o zero en la columna n_baldes de un total de 731156
Hay 0 valores negativos o zero en la columna t_aculatamiento de un total de 731156
Hay 15508 valores negativos o zero en la columna t_carga de un total de 731156
Hay 11738 valores negativos o zero en la columna t_descarga de un total de 731156
Hay 405909 valores negativos o zero en la columna t_espera_pala de un total de 731156
Hay 363615 valores negativos o zero en la columna t_cola_camion de un total de 731156
Hay 0 valores negativos o zero en la columna t_a_pala de un total de 731156
Hay 0 valores negativos o zero en la columna t_viaje_descarga de un total de 731156
Hay 3366 valores negativos o zero en la columna t_ciclo_caex de un total de 731156
Hay 15290 valores negativos o zero en la c

#### Hay una gran cantidad de valores menores o iguales a 0 en la columna `t_espera_pala` y `t_cola_camion` por lo que removeremos estas columnas momentaneamente
#### Eliminamos las filas que contengan valores negativos y los valores nulos

In [20]:
movimiento_mina = (
    movimiento_mina.na.drop()
    .select('*')
    .where((F.col('ton_camion') > 0)
          & (F.col('t_descarga') > 0)
          & (F.col('t_carga') > 0)
          & (F.col('t_ciclo_caex') > 0)
          & (F.col('t_ciclo_pala') > 0))
    .drop('t_espera_pala', 't_cola_camion')
)

In [21]:
movimiento_mina.count()

445198

In [22]:
expression = [F.round(F.mean(col), 0).alias(col) for col in movimiento_mina.columns]
mov_historico_hora = (
    movimiento_mina.select('*')
    .groupBy(F.year(F.col('Dumping')).alias('y'), 
             F.month(F.col('Dumping')).alias('m'), 
             F.dayofmonth(F.col('Dumping')).alias('d'), 
             F.hour(F.col('Dumping')).alias('h'))
    .agg(*expression)
    .withColumn('tmp', F.concat(F.col("y"), F.lit("-"), F.col('m'), F.lit("-"), F.col('d'), F.lit(" "), F.col('h'), F.lit(':00:00')))
    .withColumn('date', F.to_timestamp(F.col('tmp')))
    .drop('y', 'm', 'd', 'h', 'tmp', 'CAEX', 'PALA', 'Loading', 'Dumping')
    .orderBy('date')
)
mov_historico_hora.show(30, False)

+----------+--------+---------------+-------+----------+--------+----------------+------------+------------+-------------+-------------+----------+----------+-------------------+
|ton_camion|n_baldes|t_aculatamiento|t_carga|t_descarga|t_a_pala|t_viaje_descarga|t_ciclo_caex|t_ciclo_pala|d_viaje_vacio|d_viaje_lleno|d_eq_vacio|d_eq_lleno|date               |
+----------+--------+---------------+-------+----------+--------+----------------+------------+------------+-------------+-------------+----------+----------+-------------------+
|293.0     |4.0     |48.0           |99.0   |70.0      |398.0   |710.0           |1325.0      |147.0       |3711.0       |3411.0       |4099.0    |5409.0    |2020-12-31 21:00:00|
|306.0     |4.0     |54.0           |110.0  |59.0      |623.0   |935.0           |1782.0      |164.0       |4596.0       |4624.0       |6276.0    |7337.0    |2020-12-31 22:00:00|
|308.0     |4.0     |56.0           |108.0  |68.0      |744.0   |1039.0          |2015.0      |164.0     

In [23]:
show(plot_rango_fecha(df=mov_historico_hora, date_column='date', x='t_ciclo_caex', y='ton_camion', fecha_inicio='2021-03-05', span=True, fecha_termino='2021-03-15', cmap=False))

### Unimos ambas bases de datos en una sola, agrupadas por año, mes, dia, hora

#### El JOIN lo haremos inner pero se van a perder 8000 filas de no datos, que despues se pueden compensar con alguna actualizacion de las bases de datos

In [24]:
# Aca creo que h ya no existe
global_por_hora = ads_produccion.join(mov_historico_hora, on=(ads_produccion['fecha'] == mov_historico_hora['date']), how='inner').drop('date', 'h')
global_por_hora.count()

11664

In [25]:
#global_por_hora.toPandas().to_csv('historico_mina.csv', index=False)
global_por_hora.printSchema()

root
 |-- ton_total: double (nullable = true)
 |-- n_descargas: double (nullable = true)
 |-- n_cam: double (nullable = true)
 |-- n_shov: double (nullable = true)
 |-- ton_chancador: double (nullable = true)
 |-- ton_botadero: double (nullable = true)
 |-- descargas_botadero: double (nullable = true)
 |-- ton_chancador_1: double (nullable = true)
 |-- ton_chancador_2: double (nullable = true)
 |-- cam_chancador: double (nullable = true)
 |-- cam_botadero: double (nullable = true)
 |-- ton_alta_ley: double (nullable = true)
 |-- ton_media_ley: double (nullable = true)
 |-- ton_baja_ley: double (nullable = true)
 |-- ton_lastre: double (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- ton_camion: double (nullable = true)
 |-- n_baldes: double (nullable = true)
 |-- t_aculatamiento: double (nullable = true)
 |-- t_carga: double (nullable = true)
 |-- t_descarga: double (nullable = true)
 |-- t_a_pala: double (nullable = true)
 |-- t_viaje_descarga: double (nullable = true)
 

In [50]:
for col in global_por_hora.columns:
    try:
        lt_zero = global_por_hora.select(col).where(F.col(col) < 0).count()
        print(f'Hay {lt_zero} valores negativos en la columna {col} de un total de {global_por_hora.count()}')
    except:
        print(f'La columna {col} no se le puede aplicar esta funcion ya que es de tipo {dict(global_por_hora.dtypes)[col]}')

Hay 0 valores negativos en la columna ton_total de un total de 11664
Hay 0 valores negativos en la columna n_descargas de un total de 11664
Hay 0 valores negativos en la columna n_cam de un total de 11664
Hay 0 valores negativos en la columna n_shov de un total de 11664
Hay 0 valores negativos en la columna ton_chancador de un total de 11664
Hay 0 valores negativos en la columna ton_botadero de un total de 11664
Hay 0 valores negativos en la columna descargas_botadero de un total de 11664
Hay 0 valores negativos en la columna ton_chancador_1 de un total de 11664
Hay 0 valores negativos en la columna ton_chancador_2 de un total de 11664
Hay 0 valores negativos en la columna cam_chancador de un total de 11664
Hay 0 valores negativos en la columna cam_botadero de un total de 11664
Hay 0 valores negativos en la columna ton_alta_ley de un total de 11664
Hay 0 valores negativos en la columna ton_media_ley de un total de 11664
Hay 0 valores negativos en la columna ton_baja_ley de un total de 

In [26]:
print(f'El rango de fechas desde {global_por_hora.agg(F.min(F.col("fecha"))).collect()[0][0]} hasta {global_por_hora.agg(F.max(F.col("fecha"))).collect()[0][0]}')

El rango de fechas desde 2020-12-31 21:00:00 hasta 2022-05-11 20:00:00


In [27]:
show(plot_rango_fecha(df=global_por_hora, date_column='fecha', x='t_carga', y='ton_total', fecha_inicio='2022-02-14', span=True, fecha_termino='2022-03-14', cmap=True))

### Ahora vamos a agrupar todo por dia

In [28]:
expression = [F.round(F.mean(col), 0).alias(col) for col in global_por_hora.columns]
global_por_dia = (
    global_por_hora.select('*')
    .groupBy(F.year(F.col('fecha')).alias('y'), 
             F.month(F.col('fecha')).alias('m'), 
             F.dayofmonth(F.col('fecha')).alias('d')) 
    .agg(*expression)
    .withColumn('tmp', F.concat(F.col("y"), F.lit("-"), F.col('m'), F.lit("-"), F.col('d')))
    .withColumn('date', F.to_date(F.col('tmp')))
    .drop('y', 'm', 'd', 'tmp', 'fecha')
    .orderBy('date')
)
global_por_dia.select('date', 'ton_total', 'n_cam', 'n_shov').show(25, False)

+----------+---------+-----+------+
|date      |ton_total|n_cam|n_shov|
+----------+---------+-----+------+
|2020-12-31|22155.0  |46.0 |6.0   |
|2021-01-01|16752.0  |35.0 |5.0   |
|2021-01-02|19562.0  |40.0 |6.0   |
|2021-01-03|19331.0  |41.0 |6.0   |
|2021-01-04|15146.0  |37.0 |6.0   |
|2021-01-05|16152.0  |39.0 |5.0   |
|2021-01-06|15362.0  |39.0 |6.0   |
|2021-01-07|14863.0  |38.0 |6.0   |
|2021-01-08|18679.0  |43.0 |7.0   |
|2021-01-09|18321.0  |42.0 |7.0   |
|2021-01-10|16984.0  |40.0 |7.0   |
|2021-01-11|16859.0  |40.0 |6.0   |
|2021-01-12|16967.0  |38.0 |6.0   |
|2021-01-13|14357.0  |34.0 |5.0   |
|2021-01-14|17029.0  |38.0 |6.0   |
|2021-01-15|16281.0  |38.0 |7.0   |
|2021-01-16|13791.0  |38.0 |6.0   |
|2021-01-17|14435.0  |38.0 |5.0   |
|2021-01-18|16471.0  |38.0 |6.0   |
|2021-01-19|14965.0  |35.0 |5.0   |
|2021-01-20|15427.0  |37.0 |6.0   |
|2021-01-21|16556.0  |39.0 |7.0   |
|2021-01-22|17439.0  |39.0 |6.0   |
|2021-01-23|18102.0  |40.0 |6.0   |
|2021-01-24|16941.0  |40.0 |

In [29]:
def plot_pares_global_dia(df, x, y, date_column, fecha_inicio, fecha_termino, x_label='', y_label='', extra_info=[], span=True):
    df = (
        df.select('*')
        .filter(F.col(date_column).between(datetime.datetime.strptime(f'{fecha_inicio}', '%Y-%m-%d'),
                                            datetime.datetime.strptime(f'{fecha_termino}', '%Y-%m-%d')))
    ).toPandas()
    title = f'Rango desde {fecha_inicio} a {fecha_termino}'
    x_plot = df[x]
    y_plot = df[y]

    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column]}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, '@date{%Y-%m-%d}'), (y, '@y{int}'), (x, '@x{int}')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips, formatters={'@date': 'datetime'})
    
    p = figure(width=500, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=10, color='#2171b5', alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = title
    p.title.text_font_size = '14pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label
        
    if span:
        vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.7)
        hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.7)
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)

    return p

In [30]:
s1 = plot_pares_global_dia(df=global_por_dia, x='n_cam', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s2 = plot_pares_global_dia(df=global_por_dia, x='n_descargas', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s3 = plot_pares_global_dia(df=global_por_dia, x='ton_media_ley', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s4 = plot_pares_global_dia(df=global_por_dia, x='t_carga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s5 = plot_pares_global_dia(df=global_por_dia, x='t_descarga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s6 = plot_pares_global_dia(df=global_por_dia, x='d_viaje_vacio', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s7 = plot_pares_global_dia(df=global_por_dia, x='d_viaje_lleno', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s8 = plot_pares_global_dia(df=global_por_dia, x='t_ciclo_caex', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s9 = plot_pares_global_dia(df=global_por_dia, x='t_ciclo_pala', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [31]:
s1 = plot_pares_global_dia(df=global_por_dia, x='n_shov', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s2 = plot_pares_global_dia(df=global_por_dia, x='cam_chancador', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s3 = plot_pares_global_dia(df=global_por_dia, x='cam_botadero', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s4 = plot_pares_global_dia(df=global_por_dia, x='descargas_botadero', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s5 = plot_pares_global_dia(df=global_por_dia, x='ton_lastre', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)
s6 = plot_pares_global_dia(df=global_por_dia, x='t_viaje_descarga', y='ton_total', date_column='date', fecha_inicio='2021-01-01', fecha_termino='2022-05-01', span=True)


grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

In [32]:
expression = [F.round(F.mean(col), 0).alias(col) for col in global_por_hora.columns]
tiempos_mina_global_hora = (
    global_por_hora.select('*')
    .groupBy(F.hour(F.col('fecha')).alias('Time'))
    .agg(*expression)
    .drop('fecha')
    .orderBy('Time')
)
tiempos_mina_global_hora.toPandas().head()

,Time,ton_total,n_descargas,n_cam,n_shov,ton_chancador,ton_botadero,descargas_botadero,ton_chancador_1,ton_chancador_2,cam_chancador,cam_botadero,ton_alta_ley,ton_media_ley,ton_baja_ley,ton_lastre,ton_camion,n_baldes,t_aculatamiento,t_carga,t_descarga,t_a_pala,t_viaje_descarga,t_ciclo_caex,t_ciclo_pala,d_viaje_vacio,d_viaje_lleno,d_eq_vacio,d_eq_lleno
0,0,13176.0,43.0,37.0,6.0,4663.0,8514.0,28.0,2387.0,2275.0,13.0,24.0,147.0,4988.0,735.0,7298.0,304.0,4.0,72.0,126.0,64.0,708.0,998.0,1968.0,198.0,5241.0,4749.0,7948.0,8214.0
1,1,20791.0,68.0,44.0,7.0,7201.0,13590.0,44.0,3717.0,3485.0,17.0,32.0,227.0,7861.0,1250.0,11445.0,304.0,4.0,74.0,129.0,64.0,756.0,959.0,1983.0,203.0,5224.0,4704.0,8027.0,8136.0
2,2,21047.0,69.0,44.0,7.0,7381.0,13667.0,45.0,3774.0,3606.0,17.0,32.0,285.0,7976.0,1241.0,11540.0,304.0,4.0,73.0,129.0,64.0,703.0,965.0,1933.0,201.0,5271.0,4768.0,8037.0,8246.0
3,3,20647.0,67.0,44.0,7.0,7198.0,13449.0,44.0,3784.0,3414.0,17.0,31.0,273.0,7826.0,1310.0,11231.0,304.0,4.0,74.0,129.0,67.0,704.0,965.0,1939.0,203.0,5254.0,4757.0,8043.0,8255.0
4,4,8098.0,26.0,22.0,6.0,3053.0,5045.0,17.0,1602.0,1451.0,8.0,15.0,115.0,3377.0,456.0,4148.0,304.0,4.0,76.0,132.0,67.0,734.0,1037.0,2045.0,207.0,5197.0,4735.0,7987.0,8235.0
5,5,18054.0,59.0,40.0,7.0,6455.0,11599.0,38.0,3326.0,3129.0,16.0,28.0,223.0,6912.0,1028.0,9885.0,304.0,4.0,74.0,126.0,65.0,812.0,954.0,2031.0,200.0,5236.0,4646.0,8132.0,8021.0
6,6,19947.0,65.0,43.0,7.0,6912.0,13035.0,43.0,3597.0,3315.0,17.0,30.0,249.0,7503.0,1120.0,11071.0,304.0,4.0,73.0,128.0,65.0,737.0,956.0,1960.0,202.0,5210.0,4702.0,8012.0,8116.0
7,7,20307.0,66.0,42.0,6.0,6975.0,13332.0,44.0,3604.0,3371.0,16.0,30.0,258.0,7918.0,1188.0,10935.0,304.0,4.0,74.0,129.0,64.0,706.0,941.0,1915.0,204.0,5204.0,4660.0,8007.0,8068.0
8,8,7660.0,25.0,22.0,6.0,2791.0,4869.0,16.0,1435.0,1357.0,8.0,14.0,99.0,3114.0,419.0,4024.0,302.0,4.0,74.0,132.0,63.0,679.0,927.0,1875.0,206.0,5239.0,4775.0,7990.0,8222.0
9,9,15737.0,51.0,39.0,6.0,6123.0,9614.0,31.0,3272.0,2850.0,15.0,26.0,189.0,6311.0,1032.0,8198.0,305.0,4.0,68.0,117.0,77.0,521.0,612.0,1395.0,185.0,3982.0,3486.0,6344.0,5593.0


In [33]:
def plot_pares_global_hora(df, x, y, date_column, x_label='', y_label='', extra_info=[], span=True):
    """
    `df`  : pyspark DataFrame que contenga resultados de todas las variables agrupados por hora
    `x, y`: Nombre de columnas en `df` que se van a visualizar en el grafico
    `date_column`: Nombre de columna en `df` que contenga Hora
    `x_label, `y_label`: Nombre de los ejes en el grafico
    `extra_info`: Lista que contenga nombres de variables que se quiera conocer la informacion
    `span`: Muestra una grilla que divide el grafico en 4, respecto del max y min de cada variable, default True.
    """
    df = df.toPandas()
    x_plot = df[x]
    y_plot = df[y]
    data = {'x': x_plot, 'y': y_plot, 'date': df[date_column]}
    for idx, c in enumerate(extra_info):
        data[f'c{idx}'] = df[c]
    tooltips = [(date_column, f'@date:00 hrs'), (y, '@y'), (x, '@x')] + [(f'{c}', f'@c{idx}') for idx, c in enumerate(extra_info)]
    
    source = ColumnDataSource(data=data)
    hover_tool = HoverTool(tooltips=tooltips)
    mapper = linear_cmap(field_name='date', palette=all_palettes['PiYG'][4], low=min(df[date_column]), high=max(df[date_column]))
    color_bar = ColorBar(color_mapper=mapper['transform'], width = 20, ticker=FixedTicker(ticks=np.arange(0, 36, 6), desired_num_ticks=5), scale_alpha=0.7)
    
    p = figure(width=525, height=450, x_range=(x_plot.min(), x_plot.max()), y_range=(y_plot.min(), y_plot.max()))
    p.circle(x = 'x', y = 'y', size=25, color=mapper, alpha=0.7, source=source)
    p.title.align = 'center'
    p.title.text = f'Grafico {y_label} vs {x_label}'
    p.title.text_font_size = '12pt'
    if (x_label == '' or y_label == ''):
        p.xaxis.axis_label = x
        p.yaxis.axis_label = y
    else:
        p.xaxis.axis_label = x_label
        p.yaxis.axis_label = y_label

    vline = Span(location=(x_plot.max()+x_plot.min())*0.5, dimension='height', line_color='black', line_width=3, line_alpha=0.6)
    hline = Span(location=(y_plot.max()+y_plot.min())*0.5, dimension='width', line_color='black', line_width=3, line_alpha=0.6)
    if span:
        p.renderers.extend([vline, hline])
    p.add_tools(hover_tool)
    p.add_layout(color_bar, 'right')
    
    return p

In [34]:
show(plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='d_viaje_vacio', y='ton_total', x_label='Distancia viaje vacio', y_label='Tonelaje',
                            span=True))

In [35]:
s1 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_cam', y='ton_total', x_label='Numero Camiones', y_label='Tonelaje', span=True)
s2 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_descargas', y='ton_total', x_label='Numero Descargas', y_label='Tonelaje', span=True)
s3 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='ton_media_ley', y='ton_total', x_label='Tonelaje Ley Media', y_label='Tonelaje', span=True)
s4 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_carga', y='ton_total', x_label='Tiempo carga', y_label='Tonelaje', span=True)
s5 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_descarga', y='ton_total', x_label='Tiempo descarga', y_label='Tonelaje', span=True)
s6 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='d_viaje_vacio', y='ton_total', x_label='Distancia viaje vacio', y_label='Tonelaje', span=True)
s7 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='d_viaje_lleno', y='ton_total', x_label='Distancia viaje lleno', y_label='Tonelaje', span=True)
s8 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_ciclo_caex', y='ton_total', x_label='Tiempo ciclo CAEX', y_label='Tonelaje', span=True)
s9 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_ciclo_pala', y='ton_total', x_label='Tiempo ciclo PALA', y_label='Tonelaje', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [36]:
s1 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='n_shov', y='ton_total', x_label='Numero Palas', y_label='Tonelaje', span=True)
s2 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='cam_chancador', y='ton_total', x_label='Camiones a chancador', y_label='Tonelaje', span=True)
s3 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='cam_botadero', y='ton_total', x_label='Camiones a botadero', y_label='Tonelaje', span=True)
s4 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='descargas_botadero', y='ton_total', x_label='Descargas a botadero', y_label='Tonelaje', span=True)
s5 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='ton_lastre', y='ton_total', x_label='Tonelaje de lastre', y_label='Tonelaje', span=True)
s6 = plot_pares_global_hora(df=tiempos_mina_global_hora, date_column='Time', x='t_viaje_descarga', y='ton_total', x_label='Tiempo viaje descargas', y_label='Tonelaje', span=True)

grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

### Analisis estadistico
#### Histograma

In [37]:
global_por_hora.printSchema()

root
 |-- ton_total: double (nullable = true)
 |-- n_descargas: double (nullable = true)
 |-- n_cam: double (nullable = true)
 |-- n_shov: double (nullable = true)
 |-- ton_chancador: double (nullable = true)
 |-- ton_botadero: double (nullable = true)
 |-- descargas_botadero: double (nullable = true)
 |-- ton_chancador_1: double (nullable = true)
 |-- ton_chancador_2: double (nullable = true)
 |-- cam_chancador: double (nullable = true)
 |-- cam_botadero: double (nullable = true)
 |-- ton_alta_ley: double (nullable = true)
 |-- ton_media_ley: double (nullable = true)
 |-- ton_baja_ley: double (nullable = true)
 |-- ton_lastre: double (nullable = true)
 |-- fecha: timestamp (nullable = true)
 |-- ton_camion: double (nullable = true)
 |-- n_baldes: double (nullable = true)
 |-- t_aculatamiento: double (nullable = true)
 |-- t_carga: double (nullable = true)
 |-- t_descarga: double (nullable = true)
 |-- t_a_pala: double (nullable = true)
 |-- t_viaje_descarga: double (nullable = true)
 

In [38]:
def hist_mina(df, target='', bins=10, x_label=''):
    df = df.toPandas()
    
    hist, edges = np.histogram(df[target], bins = bins)
    plot = figure(x_range=(0, max(edges)), y_range=(0, max(hist)*1.05), plot_width=600, plot_height=450)
    plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='red', line_color='black')
    plot.xaxis.axis_label = x_label
    plot.yaxis.axis_label = 'Frecuencia'

    return plot

In [39]:
s1 = hist_mina(global_por_hora, target='n_cam', bins=10, x_label='Numero de camiones')
s2 = hist_mina(global_por_hora, target='n_descargas', bins=10, x_label='Numero de descargas')
s3 = hist_mina(global_por_hora, target='n_shov', bins=10, x_label='Numero de palas')
s4 = hist_mina(global_por_hora, target='t_carga', bins=10, x_label='Tiempo de carga')
s5 = hist_mina(global_por_hora, target='t_descarga', bins=10, x_label='Tiempo de descarga')
s6 = hist_mina(global_por_hora, target='d_viaje_vacio', bins=10, x_label='Distancia viaje vacio')
s7 = hist_mina(global_por_hora, target='d_viaje_lleno', bins=10, x_label='Distancia viaje lleno')
s8 = hist_mina(global_por_hora, target='t_ciclo_caex', bins=10, x_label='Tiempo ciclo CAEX')
s9 = hist_mina(global_por_hora, target='t_ciclo_pala', bins=10, x_label='Tiempo ciclo PALA')

grid = gridplot([[s1, s2, s3], [s4, s5, s6], [s7, s8, s9]], width=500, height=450)
show(grid)

In [40]:
s1 = hist_mina(global_por_hora, target='n_shov', bins=10, x_label='Numero de palas')
s2 = hist_mina(global_por_hora, target='cam_chancador', bins=10, x_label='Camiones a chancador')
s3 = hist_mina(global_por_hora, target='cam_botadero', bins=10, x_label='Camiones a botadero')
s4 = hist_mina(global_por_hora, target='descargas_botadero', bins=10, x_label='Descargas a botadero')
s5 = hist_mina(global_por_hora, target='ton_lastre', bins=10, x_label='Toneladas de lastre')
s6 = hist_mina(global_por_hora, target='t_viaje_descarga', bins=10, x_label='Tiempo viaje descargas')

grid = gridplot([[s1, s2, s3], [s4, s5, s6]], width=500, height=450)
show(grid)

In [41]:
def global_hora_barras(df, date_column, target='', y_label=''):
    source = ColumnDataSource(data=df.toPandas())
    tooltips = [('Hora', f'@{date_column}:00'), (y_label, f'@{target}')]
    hover_tool = HoverTool(tooltips=tooltips)

    plot = figure(x_range=(source.data[f'{date_column}'].min()-0.5, source.data[f'{date_column}'].max()+0.5), y_range=(0, source.data[f'{target}'].max()*1.05), plot_width=800, plot_height=450, x_axis_type=None)
    plot.vbar(x=date_column, bottom = 0, top=target, source=source, width=0.75)

    ticker = SingleIntervalTicker(interval=1, num_minor_ticks=1)
    xaxis = LinearAxis(ticker=ticker)
    plot.add_tools(hover_tool)
    plot.add_layout(xaxis, 'below')
    plot.xaxis.axis_label = 'Hora del dia'
    plot.yaxis.axis_label = y_label

    return plot

In [42]:
s1 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='n_cam', y_label='Numero de camiones')
s2 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='n_descargas', y_label='Numero de descargas')
s3 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='n_shov', y_label='Numero de palas')
s4 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_carga', y_label='Tiempo de carga')
s5 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_descarga', y_label='Tiempo de descarga')
s6 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='d_viaje_vacio', y_label='Distancia viaje vacio')
s7 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='d_viaje_lleno', y_label='Distancia viaje lleno')
s8 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_ciclo_caex', y_label='Tiempo ciclo CAEX')

grid = gridplot([[s1, s2], [s3, s4], [s5, s6], [s7, s8]], width=800, height=450)
show(grid)

In [43]:
s1 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_ciclo_pala', y_label='Tiempo ciclo pala')
s2 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='cam_chancador', y_label='Camiones a chancador')
s3 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='cam_botadero', y_label='Camiones a botadero')
s4 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='descargas_botadero', y_label='Descargas a botadero')
s5 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='ton_lastre', y_label='Toneladas de lastre')
s6 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_ciclo_pala', y_label='Tiempo de ciclo PALA')
s7 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='ton_media_ley', y_label='Tonelaje ley media')
s8 = global_hora_barras(df=tiempos_mina_global_hora, date_column='Time', target='t_viaje_descarga', y_label='Tiempo viaje descarga')

grid = gridplot([[s1, s2], [s3, s4], [s5, s6], [s7, s8]], width=800, height=450)
show(grid)

#### Calcular valores outliers

In [45]:
bounds = {
    c: dict(zip(["q1", "q3"], global_por_hora.approxQuantile(c, [0.25, 0.75], 0))) for c, d in zip(global_por_hora.columns, global_por_hora.dtypes) if d[1] == "double"
}

In [46]:
bounds

{'ton_total': {'q1': 13215.0, 'q3': 22029.105},
 'n_descargas': {'q1': 43.0, 'q3': 72.0},
 'n_cam': {'q1': 33.0, 'q3': 45.0},
 'n_shov': {'q1': 6.0, 'q3': 7.0},
 'ton_chancador': {'q1': 3153.21, 'q3': 8759.21},
 'ton_botadero': {'q1': 7422.3150000000005, 'q3': 15038.05},
 'descargas_botadero': {'q1': 24.0, 'q3': 49.0},
 'ton_chancador_1': {'q1': 1217.0, 'q3': 4855.0},
 'ton_chancador_2': {'q1': 594.0, 'q3': 4674.0},
 'cam_chancador': {'q1': 8.0, 'q3': 21.0},
 'cam_botadero': {'q1': 20.0, 'q3': 35.0},
 'ton_alta_ley': {'q1': 0.0, 'q3': 0.0},
 'ton_media_ley': {'q1': 3650.0, 'q3': 9299.105},
 'ton_baja_ley': {'q1': 0.0, 'q3': 918.0},
 'ton_lastre': {'q1': 6316.525, 'q3': 12698.0},
 'ton_camion': {'q1': 301.0, 'q3': 307.0},
 'n_baldes': {'q1': 4.0, 'q3': 4.0},
 't_aculatamiento': {'q1': 64.0, 'q3': 78.0},
 't_carga': {'q1': 105.0, 'q3': 142.0},
 't_descarga': {'q1': 59.0, 'q3': 67.0},
 't_a_pala': {'q1': 575.0, 'q3': 816.0},
 't_viaje_descarga': {'q1': 764.0, 'q3': 1065.0},
 't_ciclo_caex

In [47]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['q1'] = np.round(bounds[c]['q1'], 2)
    bounds[c]['q3'] = np.round(bounds[c]['q3'], 2)
    bounds[c]['min'] = np.round(bounds[c]['q1'] - (iqr * 1.5), 2)
    bounds[c]['max'] = np.round(bounds[c]['q3'] + (iqr * 1.5), 2)

In [48]:
bounds

{'ton_total': {'q1': 13215.0, 'q3': 22029.1, 'min': -6.16, 'max': 35250.26},
 'n_descargas': {'q1': 43.0, 'q3': 72.0, 'min': -0.5, 'max': 115.5},
 'n_cam': {'q1': 33.0, 'q3': 45.0, 'min': 15.0, 'max': 63.0},
 'n_shov': {'q1': 6.0, 'q3': 7.0, 'min': 4.5, 'max': 8.5},
 'ton_chancador': {'q1': 3153.21,
  'q3': 8759.21,
  'min': -5255.79,
  'max': 17168.21},
 'ton_botadero': {'q1': 7422.32,
  'q3': 15038.05,
  'min': -4001.28,
  'max': 26461.65},
 'descargas_botadero': {'q1': 24.0, 'q3': 49.0, 'min': -13.5, 'max': 86.5},
 'ton_chancador_1': {'q1': 1217.0,
  'q3': 4855.0,
  'min': -4240.0,
  'max': 10312.0},
 'ton_chancador_2': {'q1': 594.0,
  'q3': 4674.0,
  'min': -5526.0,
  'max': 10794.0},
 'cam_chancador': {'q1': 8.0, 'q3': 21.0, 'min': -11.5, 'max': 40.5},
 'cam_botadero': {'q1': 20.0, 'q3': 35.0, 'min': -2.5, 'max': 57.5},
 'ton_alta_ley': {'q1': 0.0, 'q3': 0.0, 'min': 0.0, 'max': 0.0},
 'ton_media_ley': {'q1': 3650.0,
  'q3': 9299.1,
  'min': -4823.66,
  'max': 17772.76},
 'ton_baja